# Projeto ADEL (Análise Dados Estadia em Limeira) - Webscraping das acomodações
Neste notebook, coletarei dados de acomodações anunciadas de algumas fontes. 

In [1]:
import pandas as pd

In [2]:
ss_sites = pd.read_csv('../Data/Imobiliarias/imobiliarias.csv', squeeze=True)

## Teste estático com BeautifulSoup

In [ ]:
from urllib.request import urlopen
import ssl

# This restores the same behavior as before.
context = ssl._create_unverified_context()
response = urlopen("url", context=context)
html_source = response.read()
soup = BeautifulSoup(html_source, 'html.parser')

## Coletando acomodações com Selenium

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import etree
from urllib.request import urlopen
import ssl
import hashlib
import webbrowser
import re
import time

In [64]:
def add_row_df(df, columns, row):
    row_sr = pd.Series(row, index=columns)
    row_df = pd.DataFrame([row_sr])
    df = pd.concat([row_df, df], ignore_index=True)
    return df

def hash_data(data):
    return hashlib.md5(data.encode()).hexdigest()

def goto_imovel_url(imovel_url):
    context = ssl._create_unverified_context()
    response = urlopen(imovel_url, context=context)
    html_source = response.read()
    return BeautifulSoup(html_source, 'html.parser')

def use_xpath(soup):
    return etree.HTML(str(soup))

def xpath_get_text(dom, xpath):
    texto = dom.xpath(xpath)[0].text
    return texto

In [ ]:
soup_imovel_url = goto_imovel_url('url')
dom = use_xpath(soup_imovel_url)

### Imobiliária 1/3

In [130]:
# Dataframes
df_columns = [
    'imob',
    'imovel_url',
    'codigo', 
    'nome', 
    'bairro', 
    'aluguel',
    'condominio',
    'area',
    'quartos',
    'banheiros',
    'vaga_garagem',
    'descricao'
]
df_acomodacoes = pd.DataFrame(data=None, columns=df_columns)
df_acomodacoes_hashcode = pd.DataFrame(data=None, columns=df_columns)

site_index = 0

# Hide chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

driver = webdriver.Chrome(options=chrome_options)
driver.get(ss_sites[site_index])

# Acessar página de kitnets
time.sleep(3)
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[3]/div/div').click()
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[3]/div/ul/li[11]/a').click()
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[7]/a/div').click()

# Coletar os dados das kitnets
time.sleep(3)
pag = 1

# Avançar nas páginas
try:
    while True:
        print()
        print('-'*50)
        print('Página {}'.format(pag))
        print('-'*50)
        
        # Coletar dados estaticamente com BeautifulSoup
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        soup_acomodacoes = soup.find_all('div', {'class': 'list__card jetgrid__col--6 jetgrid__col--md-8 jetgrid__col--sm-12 jetgrid__col--xs-24'})
        
        # Coletar dados de cada anúncio
        for num_acomodacao, acomodacao in enumerate(soup_acomodacoes):
            imovel_url = ss_sites[site_index] + acomodacao.find('a')['href']
            cod = acomodacao.find('p', {'class': 'list__reference'}).get_text()
            nome = acomodacao.find('p', {'class': 'list__type'}).get_text()
            bairro = acomodacao.find('p', {'class': 'list__address'}).get_text()
            aluguel = acomodacao.find('div', {'class': 'list__price'}).get_text().strip()
            print(f'Acomodação: {num_acomodacao+1}')
            print(bairro)
            print(aluguel)
            
            # Pegar os demais dados
            soup_imovel_url = goto_imovel_url(imovel_url)
            time.sleep(3.5)
            #dom = use_xpath(soup_imovel_url)
            
            # Adicionar manualmente
            webbrowser.open_new_tab(imovel_url)
            
            condominio = float(input('Digite o valor do condomínio: '))
            area = int(input('Digite a área: '))
            num_quartos = int(input('Digite o número de quartos: '))
            num_banheiros = int(input('Digite o número de banheiros: '))
            vaga_garagem = input('Digite se há vaga de garagem: ')
            descricao = soup_imovel_url.find('p', {'class': 'card__text'}).get_text()
            
            # Adicionar registro
            df_acomodacoes = add_row_df(df_acomodacoes, df_columns,
                                                   [ss_sites[site_index], 
                                                    imovel_url,
                                                    cod,
                                                    nome, 
                                                    bairro,
                                                    aluguel,
                                                    condominio,
                                                    area,
                                                    num_quartos,
                                                    num_banheiros,
                                                    vaga_garagem,
                                                    descricao])
            df_acomodacoes_hashcode = add_row_df(df_acomodacoes_hashcode, df_columns,
                                                            [hash_data(ss_sites[site_index]),
                                                             hash_data(imovel_url),
                                                             hash_data(cod), 
                                                             hash_data(nome), 
                                                             bairro,
                                                             aluguel,
                                                             condominio,
                                                             area,
                                                             num_quartos,
                                                             num_banheiros,
                                                             vaga_garagem,
                                                             descricao])
            print('*'*30)
        
        # Parar quando não encontrar próxima página
        if driver.find_element_by_xpath('/html/body/main/div[5]/section/div[3]/div[2]/div[9]/div/div/a[2]').get_attribute('href') == None:
            break
            
        # Avançar para a próxima página
        driver.find_element_by_xpath('/html/body/main/div[5]/section/div[3]/div[2]/div[9]/div/div/a[2]').click()
        pag += 1
        time.sleep(3)
        
except:
    print('Fim')


--------------------------------------------------
Página 1
--------------------------------------------------
Acomodação: 1
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.000,00
Digite o valor do condomínio: 0
Digite a área: 0
Digite o número de quartos: 1
Digite o número de banheiros: 1
Digite se há vaga de garagem: Não
******************************
Acomodação: 2
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.000,00
Digite o valor do condomínio: 0
Digite a área: 0
Digite o número de quartos: 1
Digite o número de banheiros: 1
Digite se há vaga de garagem: Não
******************************
Acomodação: 3
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 950,00
Digite o valor do condomínio: 0
Digite a área: 0
Digite o número de quartos: 1
Digite o número de banheiros: 1
Digite se há vaga de garagem: 1
******************************
Acomodação: 4
Chacara Antonieta - Limeira/SP
Locação:  R$ 850,00
Digite o valor do condomínio: 0
Digite a área: 30
Digite o núm

### Imobiliária 2/3

In [110]:
soup_test = goto_imovel_url('url')
dom_test = use_xpath(soup)

In [133]:
def scroll_command():
    js_scroll_command = "function getElementByXpath(path) {return document.evaluate(path, document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;}"
    js_scroll_command += "\nconst theElement = getElementByXpath('/html/body/main/section[1]/div[2]/div')"
    #js_scroll_command += "\nconst scrollToBottom = (node, scroll_step) => {node.scrollTo(0, 2000 * scroll_step);}"
    js_scroll_command += "\nconst scrollToBottom = (node, scroll_step) => {node.scrollTop = node.scrollHeight;}"
    js_scroll_command += f"\nscrollToBottom(theElement);"
    return js_scroll_command

In [137]:
site_index = 1

# Hide chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

#driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(ss_sites[site_index])

######################################
# Acessar página de kitnets
######################################
time.sleep(3)
# Select tipo
driver.find_element_by_xpath('/html/body/main/section[1]/div/div/div/div/div/div/form/div[1]/div[5]/div/div/button').click()
# kitnet
driver.find_element_by_xpath('/html/body/main/section[1]/div/div/div/div/div/div/form/div[1]/div[5]/div/div/div/ul/li[7]/a').click()
# Botão search
driver.find_element_by_id('submit-main-search-form').click()

# Coletar os dados das kitnets
time.sleep(3)
pag = 1
num_imoveis = 0
anuncios_encontrados = 0
anuncios = ''

# Coletar dados da página estaticamente com BeautifulSoup
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')
time.sleep(2)

# Coletar o número de imóveis encontrados
num_imoveis = soup.find('section', {'class': 'page-busca-resultado'}).find('h3').get_text()
num_imoveis = re.findall('[0-9]+', num_imoveis)
num_imoveis = int(num_imoveis[0])
print(f'num_imoveis: {num_imoveis}')

# Dar scroll até encontrar todos os imóveis
while anuncios_encontrados < num_imoveis:
    print()
    print('-'*50)
    print('Página {}'.format(pag))
    print('-'*50)
    pag += 1
    
    # Dar scroll
    driver.execute_script(scroll_command())
    time.sleep(2)
    
    # Recarregar página do soup
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(3)
    
    # Coletar os anuncios exibidos
    anuncios = soup.find('div', {'id': 'imovel-boxes'}).find_all('div', {'class': 'col-xs-12 imovel-box-single'})
    anuncios_encontrados = len(anuncios)
    
    print(f'anuncios_encontrados : {anuncios_encontrados}')
    print(anuncios[0].find('div', {'class': 'titulo-anuncio'}).find_all('p')[-1].get_text())
    print(anuncios[-1].find('div', {'class': 'titulo-anuncio'}).find_all('p')[-1].get_text())

num_imoveis: 43

--------------------------------------------------
Página 1
--------------------------------------------------
anuncios_encontrados : 24
Cód.: 36639
Cód.: 11199

--------------------------------------------------
Página 2
--------------------------------------------------
anuncios_encontrados : 36
Cód.: 36639
Cód.: 11183

--------------------------------------------------
Página 3
--------------------------------------------------
anuncios_encontrados : 43
Cód.: 36639
Cód.: 51101


In [138]:
# Coletar dados de cada anúncio
for num_acomodacao, acomodacao in enumerate(anuncios):
    imovel_url = acomodacao.find('h3', {'class': 'titulo-grid'}).find('a')['href']
    cod = acomodacao.find('div', {'class': 'titulo-anuncio'}).find_all('p')[-1].get_text()
    nome = acomodacao.find('h3', {'class': 'titulo-grid'}).find('a').get_text()
    bairro = acomodacao.find('div', {'class': 'titulo-anuncio'}).find_all('a')[-1].get_text()
    aluguel = acomodacao.find('span', {'class': 'thumb-price'}).get_text()
    print(f'Acomodação: {num_acomodacao+1}')
    print(bairro)
    print(aluguel)
    
    # Pegar os demais dados
    soup_imovel = goto_imovel_url(imovel_url)
    time.sleep(1)
    
    soup_amenities = soup_imovel.find('div', {'class': 'property-amenities'})
    condominio = 0 # Sem dados
    try:
        area = soup_amenities.find('div', {'id': 'amenity-area-privativa'}).find('span').get_text()
    except:
        webbrowser.open_new_tab(imovel_url)
        area = int(input('Digite a área: '))
    try:
        num_quartos = soup_amenities.find('div', {'id': 'amenity-dormitorios'}).find('span').get_text()
    except:
        webbrowser.open_new_tab(imovel_url)
        num_quartos = int(input('Digite o número de quartos: '))
    try:
        num_banheiros = soup_amenities.find('div', {'id': 'amenity-banheiros'}).find('span').get_text()
    except:
        webbrowser.open_new_tab(imovel_url)
        num_banheiros = int(input('Digite o número de banheiros: '))
    vaga_garagem = 'null' # Sem dados
    descricao = soup_imovel.find('section', {'id': 'clb-descricao'}).find('div', {'class': 'col-xs-12 col-sm-12 col-md-7 col-lg-8'}).find('p').get_text()
    
    # Adicionar registro
    df_acomodacoes = add_row_df(df_acomodacoes, df_columns,
                                           [ss_sites[site_index], 
                                            imovel_url,
                                            cod,
                                            nome, 
                                            bairro,
                                            aluguel,
                                            condominio,
                                            area,
                                            num_quartos,
                                            num_banheiros,
                                            vaga_garagem,
                                            descricao])
    df_acomodacoes_hashcode = add_row_df(df_acomodacoes_hashcode, df_columns,
                                                    [hash_data(ss_sites[site_index]),
                                                     hash_data(imovel_url),
                                                     hash_data(cod), 
                                                     hash_data(nome), 
                                                     bairro,
                                                     aluguel,
                                                     condominio,
                                                     area,
                                                     num_quartos,
                                                     num_banheiros,
                                                     vaga_garagem,
                                                     descricao])
    print('*'*60)

Acomodação: 1
Jardim Cidade Universitária I - Limeira/Sp 
Sob consulta
************************************************************
Acomodação: 2
Jardim Morro Azul - Limeira/Sp 
R$ 500,00
************************************************************
Acomodação: 3
Jardim Morro Azul - Limeira/Sp 
R$ 700,00
************************************************************
Acomodação: 4
Jardim Cidade Universitária I - Limeira/Sp 
R$ 700,00
************************************************************
Acomodação: 5
Jardim Morro Azul - Limeira/Sp 
R$ 700,00
************************************************************
Acomodação: 6
Jardim São Paulo - Limeira/Sp 
R$ 780,00
************************************************************
Acomodação: 7
Jardim Cidade Universitária I - Limeira/Sp 
R$ 780,00
Digite o número de quartos: 1
Digite o número de banheiros: 1
************************************************************
Acomodação: 8
Jardim Cidade Universitária I - Limeira/Sp 
R$ 800,00
*************

### Imobiliária 3/3

In [194]:
soup_test = goto_imovel_url('url')
dom_test = use_xpath(soup)

In [245]:
site_index = 2

# Hide chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

#driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(ss_sites[site_index])

# Acessar página de kitnets
time.sleep(3)
driver.find_element_by_xpath('//*[@id="busca1"]/form/div/div[1]/select').click()
driver.find_element_by_xpath('//*[@id="busca1"]/form/div/div[1]/select/option[9]').click()
driver.find_element_by_xpath('//*[@id="busca1"]/form/input[3]').click()

# Coletar os dados das kitnets
time.sleep(3)
pag = 1

# Avançar nas páginas
try:
    while True:
        print()
        print('-'*50)
        print('Página {}'.format(pag))
        print('-'*50)
        
        # Coletar dados estaticamente com BeautifulSoup
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        soup_acomodacoes = soup.find_all('div', {'class': 'imovel'})
        
        # Coletar dados de cada anúncio
        for num_acomodacao, acomodacao in enumerate(soup_acomodacoes):
            imovel_url = ss_sites[site_index] + acomodacao.find('div', {'class': 'imovelBotao'}).find('a')['href']
            cod = acomodacao.find('div', {'class': 'imovelChave'}).get_text().split('|')[0]
            nome = acomodacao.find('div', {'class': 'imovelInfo'}).find('h2').get_text().split('|')[0]
            bairro = acomodacao.find('div', {'class': 'imovelInfo'}).find('h2').get_text().split('|')[-1]
            aluguel = acomodacao.find('div', {'class': 'imovelChave'}).get_text().split('|')[-1].split()[-1]
            
            num_quartos = acomodacao.find('div', {'class': 'imovelIcones'}).find('div').get_text()
            print(f'Acomodação: {num_acomodacao+1} de {len(soup_acomodacoes)}')
            print(bairro)
            print(aluguel)
            
            # Pegar os demais dados
            soup_imovel = goto_imovel_url(imovel_url)
            time.sleep(1)
            
            condominio = soup_imovel.find('div', {'id': 'imovelExtra'}).get_text()
            area = soup_imovel.find('div', {'class': 'bloco'}).find_all('li')[0].get_text()
            # Numero de quartos e vagas de garagem
            num_banheiros = ''
            vaga_garagem = ''
            for i in range(len(soup_imovel.find('div', {'id': 'imovelIcones'}).find_all('div'))):
                if soup_imovel.find('div', {'id': 'imovelIcones'}).find_all('div')[i].find('span').get_text() == 'Banheiro':
                    num_banheiros = soup_imovel.find('div', {'id': 'imovelIcones'}).find_all('div')[i].find('strong').get_text()
                if soup_imovel.find('div', {'id': 'imovelIcones'}).find_all('div')[i].find('span').get_text() == 'Vaga':
                    vaga_garagem = soup_imovel.find('div', {'id': 'imovelIcones'}).find_all('div')[i].find('strong').get_text()
            if num_banheiros == '':
                webbrowser.open_new_tab(imovel_url)
                num_banheiros = int(input('Digite o número de banheiros: '))
            if vaga_garagem == '':
                vaga_garagem = 0 # Gabarito

            descricao = soup_imovel.find('div', {'id': 'imovelDescricao'}).find('p').get_text()
            for desc in soup_imovel.find('div', {'id': 'imovelDescricao'}).find_all('li'):
                descricao += ("\n" + desc.get_text())
            
            # Adicionar registro
            df_acomodacoes = add_row_df(df_acomodacoes, df_columns,
                                                   [ss_sites[site_index], 
                                                    imovel_url,
                                                    cod,
                                                    nome, 
                                                    bairro,
                                                    aluguel,
                                                    condominio,
                                                    area,
                                                    num_quartos,
                                                    num_banheiros,
                                                    vaga_garagem,
                                                    descricao])
            df_acomodacoes_hashcode = add_row_df(df_acomodacoes_hashcode, df_columns,
                                                            [hash_data(ss_sites[site_index]),
                                                             hash_data(imovel_url),
                                                             hash_data(cod), 
                                                             hash_data(nome), 
                                                             bairro,
                                                             aluguel,
                                                             condominio,
                                                             area,
                                                             num_quartos,
                                                             num_banheiros,
                                                             vaga_garagem,
                                                             descricao])
            
            print('*'*50)
        
        # Parar quando não encontrar próxima página
        if driver.find_element_by_xpath('//*[@id="pagenav"]/nav/ul/li[5]/a').get_attribute('href') == None:
            break
            
        # Avançar para a próxima página
        driver.find_element_by_xpath('//*[@id="pagenav"]/nav/ul/li[5]/a').click()
        pag += 1
        time.sleep(2)
        
except:
    print('Fim')


--------------------------------------------------
Página 1
--------------------------------------------------
Acomodação: 1 de 10
 JD. PAULISTA
R$950,00
**************************************************
Acomodação: 2 de 10
 CIDADE UNIVERSITARIA
R$950,00
Digite o número de banheiros: 1
**************************************************
Acomodação: 3 de 10
 CIDADE UNIVERSITARIA
R$950,00
**************************************************
Acomodação: 4 de 10
 JD. PAULISTA
R$950,00
**************************************************
Acomodação: 5 de 10
 JD. PAULISTA
R$950,00
**************************************************
Acomodação: 6 de 10
 JD. PAULISTA
R$950,00
**************************************************
Acomodação: 7 de 10
 JD. PAULISTA
R$950,00
**************************************************
Acomodação: 8 de 10
 JD. PAULISTA
R$950,00
**************************************************
Acomodação: 9 de 10
 CIDADE UNIVERSITARIA
R$1.000,00
***********************************

In [248]:
df_acomodacoes.to_csv('../Data/Imobiliarias/acomodacoes.csv', index=False)
df_acomodacoes_hashcode.to_csv('../Data/acomodacoes_hashcode.csv', index=False)

In [247]:
df_acomodacoes_hashcode.shape

(80, 12)

In [246]:
df_acomodacoes_hashcode

,imob,imovel_url,codigo,nome,bairro,aluguel,condominio,area,quartos,banheiros,vaga_garagem,descricao
0,c7234506476bbf0aff48eda764ff9eba,475b51cf4a3bf4f29cb2414680851193,57544d6162644c4cec89720f0d0fb27b,245d9e8bab947ecf1ee8867fe9597ab2,CIDADE UNIVERSITARIA,"R$1.000,00","\n\n + Condomínio R$0,00 i\...",35 M2,1,1,0,Kitnet próxima à Unicamp (FCA) contendo quarto...
1,c7234506476bbf0aff48eda764ff9eba,f2ba6e963c17f9aff690f767c73a7f87,00f33a85f39b78148f5941eac174acd2,245d9e8bab947ecf1ee8867fe9597ab2,CIDADE UNIVERSITARIA,"R$1.000,00","\n\n + Condomínio R$0,00 i\...",35 M2,1,1,0,Kitnet próxima à Unicamp (FCA) contendo quarto...
2,c7234506476bbf0aff48eda764ff9eba,047b2a430e6e4409268af5251b53d3e1,b62fad32adf34709c628e9833d66b8df,245d9e8bab947ecf1ee8867fe9597ab2,CIDADE UNIVERSITARIA,"R$1.000,00","\n\n + Condomínio R$0,00 i\...",35 M2,1,1,0,Kitnet próxima à Unicamp (FCA) contendo quarto...
3,c7234506476bbf0aff48eda764ff9eba,e3b2a519bc4e229ab0444cb144c3c407,2d7097bf091516c6952950802bd49019,245d9e8bab947ecf1ee8867fe9597ab2,CIDADE UNIVERSITARIA,"R$1.000,00","\n\n + Condomínio R$0,00 i\...",35 M2,1,1,0,Kitnet próxima à Unicamp (FCA) contendo quarto...
4,c7234506476bbf0aff48eda764ff9eba,71edf6c4e8ad2dee47bf8c9b275c022a,12d0456fa8a537ce311abb5ea43c4b03,245d9e8bab947ecf1ee8867fe9597ab2,CIDADE UNIVERSITARIA,"R$1.000,00","\n\n + Condomínio R$0,00 i\...",35 M2,1,1,0,Kitnet próxima à Unicamp (FCA) contendo quarto...
...,...,...,...,...,...,...,...,...,...,...,...,...
75,53dd1202c5ef8ce3878ffbd4b3c79bd2,bc979805840e32484e26d053a90dcaa2,13a10985cb92f1112ce6894b29a575e7,c802e68178774c3218e92187307f113a,Chacara Antonieta - Limeira/SP,"Locação: R$ 850,00",0.0,30,1,1,1,"CAMA, GUARDA-ROUPA, GELADEIRA, BANCADA, TANQU..."
76,53dd1202c5ef8ce3878ffbd4b3c79bd2,3d85eb76edfcae9812243e42f4ac9f37,a92a6e35acca298c33b9d2c83bde3505,c802e68178774c3218e92187307f113a,Chacara Antonieta - Limeira/SP,"Locação: R$ 850,00",0.0,30,1,1,Sim,"CAMA, GUARDA-ROUPA, GELADEIRA, BANCADA, TANQU..."
77,53dd1202c5ef8ce3878ffbd4b3c79bd2,ccfdb4580238d6884bc63e28ddf3223e,4b4adce0a87af0846fda6bdbe38d670a,c802e68178774c3218e92187307f113a,Jardim Cidade Universitaria I - Limeira/SP,"Locação: R$ 950,00",0.0,0,1,1,1,"KITNET CONTENDO FOGÃO, GELADEIRA, ARMÁRIO EMBU..."
78,53dd1202c5ef8ce3878ffbd4b3c79bd2,ac4a311e5c7343b1439e9bbc212e3e89,57c6964ba64836c81a7a7a782a4d0eb1,c802e68178774c3218e92187307f113a,Jardim Cidade Universitaria I - Limeira/SP,"Locação: R$ 1.000,00",0.0,0,1,1,Não,1 QUARTO COM ARMÁRIO E AR CONDICIONADO;1 SALA ...
